In [81]:
import os
import openai
import subprocess
from dotenv import load_dotenv
import json
import math
import pandas

def execute_python(file_path):
    # Run the Python file
    result = subprocess.run(['python', file_path], capture_output=True, text=True)

    # Check if the execution was successful
    if result.returncode == 0:
        # Get the output from the executed file
        output = result.stdout.strip()
        #print("Output:", output)
    else:
        # Print the error message if the execution failed
        output = result.stderr.strip()
        #print("Error:", error)
    return output

def get_response_from_gpt4(sys, usr, assist, tmp=0):
    response = openai.ChatCompletion.create(
    model="gpt-4", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
    temperature = tmp,
    messages=[
        {"role": "system", "content": sys},
        {"role": "user", "content": usr},
        {"role": "assistant", "content": assist}
    ]    
    
    )
    #res = (response['choices'][0]['message']['content'])
    return response


def get_response_from_gpt4_funct(sys, usr, assist, func, tmp=0):
    response = openai.ChatCompletion.create(
    model="gpt-4-0613", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
    #model="gpt-3.5-turbo",
    temperature = tmp,
    messages=[
        {"role": "system", "content": sys},
        {"role": "user", "content": usr},
        {"role": "assistant", "content": assist}
    ],
    functions = func
    )
    return response



querystr = """obtain the data and build a fish capture monthly data frame containing years 1986 to 2012
The expected columns are: date in the format MM/dd/yyyy, 
Total (the total dollar captured) which is a number and Wheight representing the wheight of 
the capture which is a number. Every row has the date of the first day of the month.

"""


# key01 = """ Fish capture weekly data containing years 1986 to 2012
# The columns are: date in the format MM/dd/yyyy, 
# Total capture value which is a number and Wheight representing the wheight of 
# the capture whic is a number. Every row corresponds to a week data.
# """
# available_data_01 = {

#     "name": "csv_fish_all_data",
#     "type" : "csv",
#     "route": "data/catfishall.csv",
#     "query": "",
#     "key": key01,
#     "value": -1
# }

requested_data = {
    "dataframe" : "Fish_all_data",
    "query": querystr,
    "file_path": "",
    "file_path_type" : "Pandas data frame",
    "content": "",
    "value": ""
}


key01 = """Fish capture monthly data containing years 2000 to 2012
The columns are: date in the format MM/dd/yyyy and 
Total which is a number. 
Every row has the date of the first day of the month.
"""
available_data_01 = {

    "dataframe": "data_from_2000_on",
    "file_path": "data/catfich-from_2000.csv",
    "file_path_type" : "csv",   
    "query": "",
    "content": key01,
    "value": -1
}

key02 = """ Fish capture monthly data containing years 1986 to 1999
The columns are: date in the format MM/dd/yyyy and  
Total which is a number. 
Every row has the date of the first day of the month.
"""
available_data_02 = {
    "dataframe" : "fish_from_1986_up_tp_1999",
    "file_path": "data/catfich-until1999.csv",
    "file_path_type" : "csv",   
    "query": "",
    "content": key02,
    "value": -1
}

key03 = """ Fish capture monthly data containing all years years 1986 to 2012
The columns are: date in the format MM/dd/yyyy and
Wheight which is a number. 
Every row has the date of the first day of the month.
"""
available_data_03 = {
    "dataframe" : "fish_containing_wheight",
    "file_path": "data/catfish-Allyears-onlywheight.csv",
    "file_path_type" : "csv",   
    "query": "",
    "content": key03,
    "value": -1
}

metadata_lst = [    
    json.dumps(requested_data),
    json.dumps(available_data_01),
    json.dumps(available_data_02),
    json.dumps(available_data_03)
]

load_dotenv("C:\Projrcts\Python\MSJupiter\myenv.env")
api_key = os.getenv("API_KEY")
openai.api_key = api_key

In [82]:
sys = """ You are a good data analyst and have a good understanding 
about tables, dataframes, csv and data structure in general
"""
my_string = " ".join(metadata_lst)
usr = " You have to consider the elements in the following list " + my_string + """ 
 The first element of the list is the dataframe thet we want to construct, the others  
 other data sets that could be used to build the the first one. You have to consider the content of
 each  data set to determine if it is relevant or not to build the dataframe specified in the first
  element of the list"""

assist = " Consider the following query: " + querystr + """ Use this query to find the provided
 list which is the most relevant to build the dataframe specified in the query. For that compare eqch element 
 of the list comparing the content attribute with the provided query and the content value of the first element of the list,
 then calculate a value between 0 and 5 where 5 is the most valued one. Provide me the list of the dataframe 
 and value attributes of each element. Remember the first element is the one we want to build so the values have to be
 calculated for the second element on. Please provide the result in a Json format sorting each elemnt
 descending by value. The Json format should like this:
 [
  {"dataframe": 'The dataframe name',"file_path":'the file_path',"file_path_type":'file_path_type', "value": ?}, {...}
 ]
 Copy the dataframe from the dataframe attribute in the provided list. Copy the file_path from the file_path 
 attribute in the provided list. Copy the file_path_type from the file_path_type attribute in the provided list.
 Do not provide any aditional explanation, Only the Json answer that I requested.
"""
response = get_response_from_gpt4(sys, usr, assist)
response_message = response["choices"][0]["message"]
cont = response_message["content"]
result_table = json.loads(str(cont))
ref1 = json.dumps(requested_data)
max_element = json.dumps(max(result_table, key=lambda x: x['value']))
print(cont)

# Construct the Python function 
sys = """You are an outstanding data analyst. You will be able to write a Python 
script to perform a required data manipulation. You can also use pandas to read  
a csv file into a data frame and compute differents dataframe operations. 
The task you are expected to do is to use different sources of data to create the requested final dataframe.
"""
usr = """ You will receive a query and two reference metadata describing two different dataframes. 
The query contains the expected final result, and the reference has the following 
attributes: dataframe: the name of the dataframe to be created, query: the description of the expected final result, 
file_path: the name of the file to be read into a dataframe, the file_path_type: typically csv, 
content: describing the content of that dataframe, value: you can ignore this attribute. 
Analyze the following query and the two provided references to dataframe. The firstreference 'reference1' 
is the description of the resulting dataframe that you need to build, the second 'reference2' 
is the reference to the dataframe  that you have to used to build the datafram described in the query 
attribute of'reference1'. If file_path_type = csv then you have to read it first into a dataframe from 
the external csv file provided in file_path. To read any file use the 'file_path' attribute to get 
the path for the file. For example, if the file_path attribute is data/myfile.csv then use this path 
in the read_csv command like pd.read_csv(data/myfile.csv). Never use the dataframe attribute as the 
path name for the external file. If the file_path attribute is blank, then this is a new dataframe that 
needs to be created. To name the dataframes use the attribute 'dataframe' in the references. 
For example, if the attribute 'dataframe' is 'myname' then the dataframe should be named as such. 
Generate a Python script and use it as a parameter for the function execute_python.  """
assist = "query: " + querystr + ", reference1: " + ref1 + ", reference2: " + max_element 
assist = assist + """. Make sure you comply with all the instructions given above. 
Specially make sure to use the 'file_path' attribute to get the path name to any data file to be 
loaded with the pandas command read_csv. Do not confuse the dataframe attribute with the file_path. 
The dataframe attribute only contains the name of the dataframe to be created not the file name to be used 
in any read_csv command. For that you must use the content specified in the file_path attribute. 
Very important don't make any unnecesary calculations, analyze very 
careful the 'content' attribute to understand the data contained in every dataframe in order to plan the required 
steps and calculations to be included in the Python script.
"""

func = [
            {
                "name": "execute_python",
                "description": "Execute the Python code generated by GPT",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "pythonscript": {
                            "type": "string",
                            "description": "The Python script containing the computation and workflow generated by GPT. ",
                        }
                    },
                    "required": ["pythonscript"],
                },
            }
        ]  
response1 = get_response_from_gpt4_funct(sys, usr, assist, func)
response_message = response1["choices"][0]["message"]

if response_message.get("function_call"):
    function_name = response_message["function_call"]["name"]
    print("function_name:", function_name)
    text = response_message["function_call"]["arguments"]
    #function_args = json.loads(response_message["function_call"]["arguments"])
    # Write the file with the generated script
    #text = function_args['pythonscript']
    lines = text.split('\n')
    # Write the lines to the file
    with open('data/add_numbers.py', 'w') as file:
        for line in lines:
            file.write(line + '\n')
        file.write('print(result)' + '\n')
    print(function_args)


[
  {"dataframe": "data_from_2000_on", "file_path": "data/catfich-from_2000.csv", "file_path_type": "csv", "value": 4},
  {"dataframe": "fish_from_1986_up_tp_1999", "file_path": "data/catfich-until1999.csv", "file_path_type": "csv", "value": 4},
  {"dataframe": "fish_containing_wheight", "file_path": "data/catfish-Allyears-onlywheight.csv", "file_path_type": "csv", "value": 5}
]
function_name: execute_python


NameError: name 'function_args' is not defined